### Hugging Face Transformers: Tasks ###
Use Hugging Face models for NLP tasks
- Text classification
- Named entity recognition
- Question - answering
- Translation
- Text generation

In [1]:
import os
import glob
import numpy as np
import pandas as pd

# PyTorch packages
import torch
import torch.nn as nn

# Hugging Face
from transformers import pipeline
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import set_seed

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import nlptools as nlpt
print(f'NLP Tools package version:  {nlpt.__version__}')
print(f'PyTorch version:            {torch.__version__}')

NLP Tools package version:  0.0.post1.dev31+gb496182.d20250107
PyTorch version:            2.6.0a0+df5bbc09d1.nv24.11


In [2]:
# Hugging Face API key
api_key = os.environ.get('API_KEY')
print(api_key)

None


In [3]:
# GPU checks
is_cuda = torch.cuda.is_available()
print(f'CUDA available: {is_cuda}')
print(f'Number of GPUs found:  {torch.cuda.device_count()}')

if is_cuda:
    print(f'Current device ID:     {torch.cuda.current_device()}')
    print(f'GPU device name:       {torch.cuda.get_device_name(0)}')
    print(f'CUDNN version:         {torch.backends.cudnn.version()}')
    device_str = 'cuda:0'
    torch.cuda.empty_cache() 
else:
    device_str = 'cpu'
device = torch.device(device_str)
print()
print(f'Device for model training/inference: {device}')

CUDA available: True
Number of GPUs found:  1
Current device ID:     0
GPU device name:       NVIDIA GeForce RTX 3070 Laptop GPU
CUDNN version:         90501

Device for model training/inference: cuda:0


### Task: Text classification ###

In [3]:
# Default model: distilbert-base-uncased-finetuned-sst-2-english 
classifier = pipeline(model = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english')
text1 = 'I love you'
text2 = 'I hate you'
outputs = classifier(text2)
display(pd.DataFrame(outputs))

Device set to use cuda:0


,label,score
0,NEGATIVE,0.999113


### Task: Named entity recognition NER ###

In [4]:
ner_tagger = pipeline('ner', aggregation_strategy='simple')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [5]:
text = 'My name is Andreas Werdich and I am working at Harvard Medical School'
outputs = ner_tagger(text)
display(pd.DataFrame(outputs))

,entity_group,score,word,start,end
0,PER,0.995936,Andreas Werdich,11,26
1,ORG,0.866248,Harvard Medical School,47,69


In [6]:
# Medical terms extractions
model_name = 'blaze999/Medical-NER'
ner_tagger = pipeline(model=model_name, aggregation_strategy='simple')

Device set to use cuda:0


In [7]:
text = 'A 48 year-old female presented with vaginal bleeding and abnormal Pap smears. Upon diagnosis of invasive non-keratinizing SCC of the cervix, she underwent a radical hysterectomy with salpingo-oophorectomy which demonstrated positive spread to the pelvic lymph nodes and the parametrium. Pathological examination revealed that the tumour also extensively involved the lower uterine segment.'
outputs = ner_tagger(text)
display(pd.DataFrame(outputs))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,AGE,0.941736,48 year-old,1,13
1,SEX,0.818373,female,13,20
2,CLINICAL_EVENT,0.853015,presented,20,30
3,BIOLOGICAL_STRUCTURE,0.944933,vaginal,35,43
4,SIGN_SYMPTOM,0.872998,bleeding,43,52
5,LAB_VALUE,0.769152,abnormal,56,65
6,DIAGNOSTIC_PROCEDURE,0.987654,Pap smears,65,76
7,DETAILED_DESCRIPTION,0.934324,invasive,95,104
8,DETAILED_DESCRIPTION,0.961545,non-keratinizing,104,121
9,DISEASE_DISORDER,0.784436,SCC,121,125


In [8]:
model_name = 'varunnagda/bert-medication'
ner_tagger = pipeline(model=model_name, aggregation_strategy='simple')

Device set to use cuda:0


In [9]:
text = 'John Doe, a 53-year-old male, was admitted to City General Hospital on 09/25/2023 with acute exacerbation of COPD and community-acquired pneumonia. He received high-flow oxygen therapy and intravenous antibiotics, which led to significant improvement in his respiratory function. By 10/01/2023, he transitioned to oral antibiotics and nasal cannula oxygen. Physical therapy sessions enhanced his lung capacity, enabling discharge on 10/05/2023 in stable condition. The discharge medications include Albuterol Inhaler, Tiotropium, a tapering course of Prednisone, Azithromycin, and Pantoprazole. John is advised to avoid lung irritants, adhere to his medication regimen, and seek medical attention if symptoms recur. Follow-up appointments are scheduled with pulmonology on 10/12/2023 and his primary care physician on 10/14/2023.'
outputs = ner_tagger(text)
display(pd.DataFrame(outputs))

,entity_group,score,word,start,end
0,Medication,0.944490,albuterol,499,508
1,Medication,0.616578,##r,515,516
2,Medication,0.952474,tiotropium,518,528
3,Medication,0.967220,prednisone,551,561
4,Medication,0.972024,azithromycin,563,575
5,Medication,0.975528,pantoprazole,581,593


### Task: Extractive Question-Answering ###

In [39]:
# Context
text = '''
Dear Amazon Customer Service,
I hope this message finds you well. I am writing to bring to your attention an issue with a recent order delivered to me. 
I ordered a web camera (Order #2378), but unfortunately, I received a different model than what I had originally ordered.
I kindly request assistance in resolving this matter. 
Could you please provide guidance on how to return the incorrect item and ensure 
the correct web camera is sent to me as soon as possible? Thank you for your prompt attention to this issue. 
I appreciate your support and look forward to resolving this matter quickly.
Best regards, Andreas Werdich
'''
# Questions
question_list = ['What was wrong with the order?', 
                 'What is the name of the person who wrote the message?',
                 'What is the order number?']

# Models to try
model_name_list = ['distilbert/distilbert-base-cased-distilled-squad',
                   'deepset/roberta-base-squad2']

In [42]:
# Run the models on the task
for model_name in model_name_list:
    print(f'model: {model_name}')
    reader = pipeline(task = 'question-answering', model=model_name)
    outputs = reader(question=question_list, context=text) 
    outputs_df = pd.DataFrame(outputs).assign(question=question_list)
    display(outputs_df)

Device set to use cuda:0


model: distilbert/distilbert-base-cased-distilled-squad


,score,start,end,answer,question
0,0.098921,211,239,I received a different model,What was wrong with the order?
1,0.799117,1,29,Dear Amazon Customer Service,What is the name of the person who wrote the m...
2,0.623703,185,189,2378,What is the order number?


model: deepset/roberta-base-squad2


Device set to use cuda:0


,score,start,end,answer,question
0,0.227721,222,274,a different model than what I had originally o...,What was wrong with the order?
1,0.008343,614,629,Andreas Werdich,What is the name of the person who wrote the m...
2,0.688542,185,189,2378,What is the order number?


### Task: text summarization ###

In [49]:
model_name_list = ['facebook/bart-large-cnn',
                   'sshleifer/distilbart-cnn-12-6']
                   
for model_name in model_name_list:
    print(f'model: {model_name}')
    summarizer = pipeline(task='summarization', model=model_name)
    max_length = 130
    min_length = 30
    summary = summarizer(text, 
                         max_length=max_length, 
                         min_length=min_length, 
                         do_sample=True)
    display(summary[0].get('summary_text'))

model: facebook/bart-large-cnn


Device set to use cuda:0


'Andreas Werdich received a different model of a web camera than what he had originally ordered. He asked Amazon Customer Service for guidance on how to return the incorrect item and ensure the correct camera is sent to him.'

model: sshleifer/distilbart-cnn-12-6


Device set to use cuda:0


' Andreas Werdich writes to Amazon Customer Service . He asks for guidance on how to return the incorrect item and ensure the correct camera is sent to him as soon as possible .'

### Task: Text generation ###

In [81]:
response = 'I am sorry that your order was mixed up'
# GPT-2 is not so good at writing a response
p_1 = f'User: {text.replace("\n", "")}. Customer service representative response: {response}'

# # Let's try a different one
p_2 = f'There was n alligator '

prompt_list = [p_1, p_2]

In [87]:
# Models to try
model_name_list = ['openai-community/gpt2',
                   'mistralai/Mistral-7B-Instruct-v0.2']

for model_name in model_name_list:
    print(model_name)
    generator = pipeline(task='text-generation', model=model_name)
    for prompt in prompt_list:
        print()
        print(f'prompt: {prompt}')
        set_seed(1334)
        outputs = generator(prompt, max_new_tokens=128, 
                            do_sample=True, return_full_text=False)
        display(outputs)

openai-community/gpt2


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



prompt: User: Dear Amazon Customer Service,I hope this message finds you well. I am writing to bring to your attention an issue with a recent order delivered to me. I ordered a web camera (Order #2378), but unfortunately, I received a different model than what I had originally ordered.I kindly request assistance in resolving this matter. Could you please provide guidance on how to return the incorrect item and ensure the correct web camera is sent to me as soon as possible? Thank you for your prompt attention to this issue. I appreciate your support and look forward to resolving this matter quickly.Best regards, Andreas Werdich. Customer service representative response: I am sorry that your order was mixed up


[{'generated_text': ". I did not receive the same item on my order and my current order. I received the item and it was delivered the same day as you would expect. Your customer service company had promised me this same item which is not only delivered but arrived in my address which is about 5 days before I received it. I am sure this is due to not having a buyer's documentation card used as a delivery method. Any additional inquiries please contact me. Thank you for your quick response.\n\nIf you would like additional guidance, I would like to contact you as soon as possible. Thank you for your prompt response."}]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



prompt: There was n alligator 


[{'generated_text': 'ichthyosaurs, and it has not had dinosaurs," said Mark Ewan Johnson, a doctoral student and former senior scientist of paleoscientists at the Ohio State University, who was not involved with the work.\n\nAn estimated 15 million years ago the dinosaurs disappeared about 5,000 feet away from one of the giants. Archaeologists find skeletons of a young giant that weighs about 500 pounds. The bones of a skeleton known as a jay can go missing in the Cretaceous period about 120,000 years ago.\n\nJohnson said there have been no documented signs of dinosaurs that lived during this era, a rarity even in'}]

mistralai/Mistral-7B-Instruct-v0.2


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-677acca0-7ab570292516dbba31973c86;c29fd0e6-abbb-4d79-9893-f9252d0f2636)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.